# Variational inference (mean field)

https://cse.buffalo.edu/faculty/mbeal/papers/beal03.pdf
https://www.cs.princeton.edu/courses/archive/fall11/cos597C/lectures/variational-inference-i.pdf
http://www.maths.usyd.edu.au/u/jormerod/JTOpapers/Ormerod10.pdf

When doing mean-field approximation, we enforce independence between groups of or all of the unobserved variables. If we (are willing to) pick conjugate priors, then it is possible to derive closed-form updates for the approximation functions $q$ of each of the model parameter groups. We can iterate through the groups to update the parameters given the current estimate of all other parameter groups.

TODO: Understand when this is a good assumption. I think whenever dependence is not strong in the first place and when dependence is strong only in places with low probability mass?



Great blog showing the general derviation of mean-field approximation: https://bjlkeng.github.io/posts/variational-bayes-and-the-mean-field-approximation/
See derivations --> 

$q_j(\theta_j) = e^{E_{m|m \neq n}[log p(\theta, X)]} \cdot \frac{1}{Z_j}$

In [1]:
import numpy as np

In [142]:
mu = 10
tau = 1/2
X = np.random.normal(mu, scale = 1/np.sqrt(tau), size = 10000)

In [143]:
print(np.mean(X))
print(np.var(X))

10.011185680581566
1.9950700561309764


TODO: Add step-by-step derivations from notes

#### Aside 
Completing the square:
https://en.wikipedia.org/wiki/Completing_the_square

$x^2 + bx +c = (x + \frac{b}{2})^2 + (c- \frac{b^2}{4})$

In [144]:
def update_mu(X, mu_0, lambda_0):
    N = len(X)
    mu = (np.sum(X) - 2*lambda_0*mu_0) / (N+lambda_0)
    return mu

In [145]:
def update_tau(X, lambda_0, alpha, beta):
    tau = (lambda_0 + len(X)) * alpha/beta
    return tau

In [146]:
def update_alpha(alpha_0, X):
    N = len(X)
    alpha_tau = alpha_0 + 0.5*(N+1)
    return alpha_tau

In [147]:
def update_beta(beta_0, mu_0, mu, tau, lambda_0, X):
    N = len(X)
    E_mu2 = 1/tau + mu**2
    beta_tau = beta_0 +0.5*lambda_0*(E_mu2-2*mu_0*mu+mu_0**2) \
                    +0.5*np.sum((X**2) - 2*mu*X +E_mu2)
    return beta_tau

In [148]:
update_beta(1,0,10,100,0,X)

10026.975877905272

In [149]:
def run_VB(start_values, hyperparameter, n_iter):
    # Initialize output array
    trace = np.empty([n_iter,4])
    # Unpack input parameters
    mu, tau, alpha, beta = (start_values[x] for x in ["mu","tau","alpha","beta"])
    X, mu_0, lambda_0, alpha_0, beta_0 = (hyperparameter[x] for x in ["X", "mu_0","lambda_0","alpha_0","beta_0"])
    
    # These variables don't depend on changing variables
    mu = update_mu(X, mu_0, lambda_0)
    alpha = update_alpha(alpha_0, X)
    
    # Iteratively update the other variables
    for it in range(n_iter):
        tau = update_tau(X, lambda_0, alpha, beta)
        beta = update_beta(beta_0, mu_0, mu, tau, lambda_0, X)
        # Save values per iteration
        trace[it,:] = (mu,tau,alpha,beta)
    
    return trace
    

In [163]:
start_values = {
    "mu":0,
    "tau":2,
    "alpha":10,
    "beta":10,
}

hyperparameter = {
    "X":X,
    "mu_0":0,
    "lambda_0":1/np.sum(np.square(X)), # Related to var(beta_OLS)?
    "alpha_0":2,
    "beta_0":1
}

In [164]:
trace = run_VB(start_values, hyperparameter, 200)

In [165]:
print(trace[-1:,:])

[[  10.01118568 5013.85760844 5002.5        9977.34756582]]


In [166]:
result = trace[-1,:]
print(round(result[0]), # Expectation of the Normal distribution for mu
      round(result[2]/result[3],3)) # Expectation of the Gamma distribution for tau
print(mu, tau)

10.0 0.501
10 0.5


## Variational Bayes for Regression

Derivation: https://arxiv.org/pdf/1310.5438v2.pdf

Comparison to frequentiest motivation: https://onlinecourses.science.psu.edu/stat857/node/155/

TODO: Everyone seems to assume $\tau_{beta} = \lambda \tau$. I see where that gets convenient in the derivation, but shouldn't it work without, too? What's the difference in meaning?

In [134]:
# Create a simple 1-D regression case (without constant)
beta_true = 2
tau_true = 1
n_obs = 10000
X = np.random.uniform(-4,4, size =n_obs)
y = beta_true*X + np.random.normal(0,1/np.sqrt(tau_true), size = len(X))

In [135]:
print(X.shape)
print(y.shape)

(10000,)
(10000,)


In [51]:
def update_alpha(alpha_0, X):
    N = len(X)
    alpha_tau = alpha_0 + 0.5*N
    return alpha_tau

In [136]:
def update_beta(beta_0, mu, tau_beta, X):
    E_mu2 = 1/tau_beta + mu**2
    beta_tau = beta_0 +0.5*(np.dot(y,y)-2*mu*np.dot(y,X)+E_mu2*np.dot(X,X.T))
    return beta_tau

In [128]:
def update_mu(X,y, alpha, beta, tau_beta_0):
    E_tau = alpha/beta
    mu =  E_tau * np.dot(y,X) / (E_tau *np.dot(X.T,X) + tau_beta_0)
    return mu

In [137]:
def update_tau_beta(X, tau_beta_0, alpha, beta):
    tau_beta = (tau_beta_0 + np.dot(X.T,X)) * alpha/beta
    return tau_beta

In [145]:
def run_VB(start_values, hyperparameter, n_iter):
    # Initialize output array
    trace = np.empty([n_iter,4])
    # Unpack input parameters
    mu, tau_beta, alpha, beta = (start_values[x] for x in ["mu","tau_beta","alpha","beta"])
    X,y, tau_beta_0, alpha_0, beta_0 = (hyperparameter[x] for x in ["X","y", "tau_beta_0","alpha_0","beta_0"])
    
    # These variables don't depend on changing variables
    alpha = update_alpha(alpha_0, X)
    
    # Iteratively update the other variables
    for it in range(n_iter):
        mu = update_mu(X,y, alpha, beta, tau_beta_0)
        tau_beta = update_tau_beta(X, tau_beta_0, alpha, beta)
        beta = update_beta(beta_0, mu, tau_beta, X)
        # Save values per iteration
        trace[it,:] = (mu,tau_beta,alpha,beta)
    
    return trace
    

In [146]:
start_values = {
    "mu":0,
    "tau_beta":2,
    "alpha":10,
    "beta":10,
}

hyperparameter = {
    "X":X,
    "y":y,
    "tau_beta_0":1,
    "alpha_0":2,
    "beta_0":1
}

In [147]:
trace = run_VB(start_values, hyperparameter, 200)

In [151]:
print(trace[-1,:])

[1.99973285e+00 5.37780795e+04 5.00200000e+03 4.94835491e+03]


In [152]:
result = trace[-1,:]
print(round(result[0]), # Expectation of the Normal distribution for mu
      round(result[2]/result[3])) # Expectation of the Gamma distribution for tau
print(beta_true, tau_true)

2.0 1.0
2 1


TODO: Extend derivation and code to multivariate case. If we assume that all beta priors are identical, then do this in matrix notation and code 